# Customize Thinking

`CognitiveWorker` provides several template methods that let you customize every stage of the "observe-think-act" cycle. By overriding these methods, you can control what the agent sees, how it reasons, which tools it can use, and how results are processed.

| Template Method | Stage | Purpose |
|-----------------|-------|----------|
| `observation()` | Observe | Add custom context before thinking |
| `build_thinking_prompt()` | Think | Customize how the prompt is assembled |
| `verify_tools()` | Act | Gate or modify tools before execution |
| `consequence()` | Post-Act | Format or summarize tool results |
| `select_tools()` | Act (DEFAULT mode only) | Custom tool selection logic |

## 1. Setup

We'll reuse the same travel planning scenario from the Agent Automa tutorial. Each section is self-contained.

In [ ]:
import os
from typing import Any, List, Optional, Tuple

_api_key = os.environ.get("OPENAI_API_KEY")
_api_base = os.environ.get("OPENAI_API_BASE")
_model_name = os.environ.get("OPENAI_MODEL_NAME")

from bridgic.core.cognitive import (
    CognitiveContext,
    CognitiveWorker,
    ThinkingMode,
    AgentAutoma,
    think_step,
    ActionStepResult,
)
from bridgic.core.agentic.tool_specs import FunctionToolSpec, ToolSpec
from bridgic.core.model.types import ToolCall
from bridgic.llms.openai import OpenAILlm

llm = OpenAILlm(api_base=_api_base, api_key=_api_key, timeout=120)


# --- Mock tools ---
def search_flights(origin: str, destination: str) -> str:
    """Search for available flights between two cities.

    Parameters
    ----------
    origin : str
        The departure city.
    destination : str
        The arrival city.
    """
    return f"Found 3 flights from {origin} to {destination}: FL100 ($300), FL200 ($450), FL300 ($280)"


def search_hotels(city: str) -> str:
    """Search for available hotels in a city.

    Parameters
    ----------
    city : str
        The city to search hotels in.
    """
    return f"Found 3 hotels in {city}: Grand Hotel ($120/night), City Inn ($80/night), Budget Stay ($50/night)"


def book_flight(flight_id: str) -> str:
    """Book a specific flight by its ID.

    Parameters
    ----------
    flight_id : str
        The flight identifier (e.g., FL100).
    """
    return f"Successfully booked flight {flight_id}. Confirmation: BK-{flight_id}-2024"


def book_hotel(hotel_name: str, nights: str) -> str:
    """Book a hotel for a specified number of nights.

    Parameters
    ----------
    hotel_name : str
        The name of the hotel to book.
    nights : str
        Number of nights to stay.
    """
    return f"Successfully booked {hotel_name} for {nights} nights. Confirmation: HBK-{hotel_name[:3].upper()}-2024"


travel_tools = [
    FunctionToolSpec.from_raw(search_flights),
    FunctionToolSpec.from_raw(search_hotels),
    FunctionToolSpec.from_raw(book_flight),
    FunctionToolSpec.from_raw(book_hotel),
]


class TravelPlanningContext(CognitiveContext):
    """Context with travel planning tools pre-loaded."""

    def __post_init__(self):
        for tool_spec in travel_tools:
            self.tools.add(tool_spec)

<div style="text-align: center; margin: 2rem 0;"><hr style="border: none; border-top: 2px solid #e2e8f0;"></div>

## 2. Override `observation()`

The `observation()` method is called before thinking. It can return custom text that gets included in the thinking context. Use it to inject situational awareness, summarize recent events, or add any domain-specific observations.

By default, `observation()` returns `None` (no custom observation).

In [ ]:
class ObservantWorker(CognitiveWorker):
    """Worker that adds custom observations before thinking."""

    async def thinking(self) -> str:
        return (
            "You are a reactive assistant. Focus on ONE step at a time. "
            "Observe the current state, determine the best next action, and execute it."
        )

    async def observation(self, context: CognitiveContext) -> Optional[str]:
        # Add a custom observation based on history
        history_len = len(context.cognitive_history)
        if history_len == 0:
            return "This is the first step. No prior actions have been taken yet."
        else:
            latest = context.cognitive_history[history_len - 1]
            status = "succeeded" if latest.status else "failed"
            return f"Previous action '{latest.content}' {status}. Total steps so far: {history_len}."


class ObservantAgent(AgentAutoma[TravelPlanningContext]):
    react = think_step(
        ObservantWorker(llm=llm, mode=ThinkingMode.FAST, verbose=True)
    )

    async def cognition(self, ctx: TravelPlanningContext):
        for _ in range(5):
            await self.react
            if ctx.finish:
                break


agent = ObservantAgent(llm=llm)
result = await agent.arun(
    goal="Search for flights from Beijing to Kunming."
)

print(f'\n- - - - - Result - - - - -')
print(result)

<div style="text-align: center; margin: 2rem 0;"><hr style="border: none; border-top: 2px solid #e2e8f0;"></div>

## 3. Override `build_thinking_prompt()`

The `build_thinking_prompt()` method assembles the final `(system_prompt, user_prompt)` pair sent to the LLM. It receives four components:

- `think_prompt` -- from your `thinking()` method
- `tools_description` -- formatted tool information
- `output_instructions` -- instructions for the output format
- `context_info` -- goal, history, and status

Override it to inject extra instructions, reorder components, or add domain-specific constraints. Here we add a budget constraint to the system prompt and use `verbose_prompt=True` to inspect the modified prompt.

In [ ]:
class BudgetConstrainedWorker(CognitiveWorker):
    """Worker that injects budget constraint into the system prompt."""

    async def thinking(self) -> str:
        return (
            "You are a planning-oriented assistant. Create a complete plan "
            "covering all steps needed to achieve the goal."
        )

    async def build_thinking_prompt(
        self,
        think_prompt: str,
        tools_description: str,
        output_instructions: str,
        context_info: str,
    ) -> Tuple[str, str]:
        # Add budget constraint to the system prompt
        budget_instruction = (
            "\n\n# Budget Constraint:\n"
            "The user has a strict budget of $500 total for flights and hotels combined. "
            "Always prefer the cheapest options. If total cost would exceed $500, "
            "warn the user in the step description."
        )

        system_prompt = f"{think_prompt}{budget_instruction}"
        if tools_description:
            system_prompt += f"\n\n{tools_description}"
        system_prompt += f"\n\n{output_instructions}"

        return system_prompt, context_info


class BudgetAgent(AgentAutoma[TravelPlanningContext]):
    plan = think_step(
        BudgetConstrainedWorker(llm=llm, mode=ThinkingMode.DEFAULT, verbose=True, verbose_prompt=True)
    )

    async def cognition(self, ctx: TravelPlanningContext):
        await self.plan


agent = BudgetAgent(llm=llm)
result = await agent.arun(
    goal="I'm in Beijing and want to go to Kunming. Plan my route and hotel."
)

print(f'\n- - - - - Result - - - - -')
print(result)

<div style="text-align: center; margin: 2rem 0;"><hr style="border: none; border-top: 2px solid #e2e8f0;"></div>

## 4. Override `verify_tools()`

The `verify_tools()` method is called after tools are matched but before they are executed. It receives a list of `(ToolCall, ToolSpec)` pairs and returns the verified list.

Use this as a **safety gate**: block certain tools based on conditions, log tool usage, or modify arguments before execution.

Here we implement a rule: booking tools (`book_flight`, `book_hotel`) are blocked unless a corresponding search has been performed first.

In [ ]:
class SafetyGateWorker(CognitiveWorker):
    """Worker that blocks booking without prior search."""

    async def thinking(self) -> str:
        return (
            "You are a reactive assistant. Focus on ONE step at a time. "
            "Observe, think, and act."
        )

    async def verify_tools(
        self,
        matched_list: List[Tuple[ToolCall, ToolSpec]],
        context: CognitiveContext,
    ) -> List[Tuple[ToolCall, ToolSpec]]:
        # Check which searches have been performed
        searched_tools = set()
        for step in context.cognitive_history:
            tool_names = step.metadata.get("tool_calls", [])
            searched_tools.update(tool_names)

        verified = []
        for tc, spec in matched_list:
            # Block booking if no prior search exists
            if tc.name == "book_flight" and "search_flights" not in searched_tools:
                print(f"[Safety Gate] BLOCKED {tc.name}: no prior flight search found.")
                continue
            if tc.name == "book_hotel" and "search_hotels" not in searched_tools:
                print(f"[Safety Gate] BLOCKED {tc.name}: no prior hotel search found.")
                continue
            verified.append((tc, spec))

        if not verified:
            # If all tools were blocked, fall back to the first search tool
            print("[Safety Gate] All tools blocked. Falling back to original matched list.")
            return matched_list

        return verified


class SafeAgent(AgentAutoma[TravelPlanningContext]):
    react = think_step(
        SafetyGateWorker(llm=llm, mode=ThinkingMode.FAST, verbose=True)
    )

    async def cognition(self, ctx: TravelPlanningContext):
        for _ in range(5):
            await self.react
            if ctx.finish:
                break


agent = SafeAgent(llm=llm)
result = await agent.arun(
    goal="Book flight FL100 from Beijing to Kunming and a hotel."
)

print(f'\n- - - - - Result - - - - -')
print(result)

<div style="text-align: center; margin: 2rem 0;"><hr style="border: none; border-top: 2px solid #e2e8f0;"></div>

## 5. Override `consequence()`

The `consequence()` method processes the raw `ActionStepResult` list from tool execution before it's stored in history. By default, it returns the raw list. Override it to format, summarize, or transform results.

This is useful for keeping history clean and readable, especially when tool results are verbose.

In [ ]:
class CleanResultWorker(CognitiveWorker):
    """Worker that formats raw tool results into clean summaries."""

    async def thinking(self) -> str:
        return (
            "You are a reactive assistant. Focus on ONE step at a time. "
            "Observe, think, and act."
        )

    async def consequence(self, action_results: List[ActionStepResult]) -> Any:
        # Format each tool result into a clean summary string
        summaries = []
        for r in action_results:
            summary = f"[{r.tool_name}] {r.tool_result}"
            summaries.append(summary)
        return " | ".join(summaries)


class CleanAgent(AgentAutoma[TravelPlanningContext]):
    react = think_step(
        CleanResultWorker(llm=llm, mode=ThinkingMode.FAST, verbose=True)
    )

    async def cognition(self, ctx: TravelPlanningContext):
        for _ in range(5):
            await self.react
            if ctx.finish:
                break


agent = CleanAgent(llm=llm)
result = await agent.arun(
    goal="Search for flights from Beijing to Kunming."
)

# Show the formatted history entries
print(f'\n- - - - - History - - - - -')
for i, step in enumerate(result.cognitive_history):
    print(f"Step {i}: {step.content}")
    print(f"  Result: {step.result}")
print(f'- - - - - End - - - - -')

<div style="text-align: center; margin: 2rem 0;"><hr style="border: none; border-top: 2px solid #e2e8f0;"></div>

## 6. Override `select_tools()`

The `select_tools()` method is only used in `DEFAULT` mode. By default, it uses an LLM call to select the appropriate tools for each step. Override it to implement custom selection logic -- for example, keyword-based matching that avoids an extra LLM call.

You can always fall back to the default LLM-based selection by calling `self._select_tools_for_step()` for cases your custom logic doesn't handle.

In [ ]:
class KeywordToolSelector(CognitiveWorker):
    """Worker that uses keyword matching for tool selection (DEFAULT mode)."""

    async def thinking(self) -> str:
        return (
            "You are a planning-oriented assistant. Create a complete plan "
            "covering all steps needed to achieve the goal."
        )

    async def select_tools(self, step_content: str, context: CognitiveContext) -> List[ToolCall]:
        step_lower = step_content.lower()

        # Simple keyword matching
        if "search" in step_lower and "flight" in step_lower:
            return [ToolCall(id="1", name="search_flights", arguments={"origin": "Beijing", "destination": "Kunming"})]
        elif "search" in step_lower and "hotel" in step_lower:
            return [ToolCall(id="1", name="search_hotels", arguments={"city": "Kunming"})]
        elif "book" in step_lower and "flight" in step_lower:
            return [ToolCall(id="1", name="book_flight", arguments={"flight_id": "FL300"})]
        elif "book" in step_lower and "hotel" in step_lower:
            return [ToolCall(id="1", name="book_hotel", arguments={"hotel_name": "Budget Stay", "nights": "2"})]
        else:
            # Fall back to LLM-based selection for unmatched cases
            print(f"[KeywordSelector] No keyword match for: {step_content}. Falling back to LLM.")
            return await self._select_tools_for_step(step_content, context)


class KeywordAgent(AgentAutoma[TravelPlanningContext]):
    plan = think_step(
        KeywordToolSelector(llm=llm, mode=ThinkingMode.DEFAULT, verbose=True)
    )

    async def cognition(self, ctx: TravelPlanningContext):
        await self.plan


agent = KeywordAgent(llm=llm)
result = await agent.arun(
    goal="I'm in Beijing and want to go to Kunming. Plan my route and book the cheapest options."
)

print(f'\n- - - - - Result - - - - -')
print(result)

## What have we learnt?

| Template Method | Default Behavior | When to Override |
|-----------------|-----------------|------------------|
| `observation()` | Returns `None` (no custom observation) | Add situational awareness, summarize recent events, inject domain context |
| `build_thinking_prompt()` | Concatenates think_prompt + tools + output instructions | Inject extra constraints, reorder prompt sections, add domain rules |
| `verify_tools()` | Returns matched list as-is | Safety gates, blocking dangerous tools, validating preconditions |
| `consequence()` | Returns raw `ActionStepResult` list | Format results for cleaner history, extract key data, summarize verbose output |
| `select_tools()` | LLM-based selection (DEFAULT mode only) | Keyword matching, rule-based selection, reducing LLM calls |